# Cleaning up and merging NBA data collected
To be used after executing scrape.py file
## Importing required libraries

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
from string import ascii_lowercase
import os
from glob import glob
pd.set_option('display.max_columns', 500)

## Creating functions to clean and merge data sets
#### MUST MAKE SURE YOU ARE IN BASE DIRECTORY WHERE ALL PROJECT FILES AND SUB-DIRECTORIES ARE

In [141]:
#os.getcwd()
back_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
os.chdir(back_dir)
os.getcwd()

'C:\\Users\\davil\\Desktop\\test'

### Function to merge csv files separated by season
___

In [143]:
def merge_csv_by_season_player(folder_from: object,
                        csv_name: object, 
                        folder_write_to: object) -> object:
    #go to folder where csv files to be merged are
    os.chdir(folder_from)
    #create file list variable that will have all merged file data
    file_list = list()
    
    #looping through files in directory to prep data than append to master dataframe
    for file in os.listdir():
        if file.endswith('.csv'):
            df_2w = pd.read_csv(file, encoding="utf-8") 
            df_2w['Season'] = file[:4]
            #split Player column into Player_Name and Player_ID
            new = df_2w["Player"].str.split("\\", expand = True)
            df_2w["Player_Name"]= new[0]
            df_2w["Player_ID"]= new[1]
            df_2w.drop(columns = ["Player"], inplace = True)
            # move the column to head of list using index, pop and insert
            cols = list(df_2w)
            cols.insert(0, cols.pop(cols.index('Player_Name')))
            cols.insert(1, cols.pop(cols.index('Player_ID')))
            cols
            # use loc to reorder
            df_2w = df_2w.loc[:, cols]
            #fill NAs
            df_2w.fillna(0, inplace=True)           
            #append to master dataframe to be written to csv                               
            file_list.append(df_2w)
    #merge all data in file_list, then create new file to write merged data to        
    merged_files = pd.concat(file_list, axis=0, ignore_index=True)
    #change to directory we are writing csv to
    back_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    os.chdir(back_dir)
    os.chdir(folder_write_to)
    #write csv
    merged_files.to_csv("all_seasons_" + csv_name)
    #go back to original directory
    os.chdir(back_dir)

### Using merge function for regular season and playoffs stats

##### Regular Season Stats

In [110]:
#Reg Season - Per Game Stats by Players
merge_csv_by_season_player(folder_from = 'season_stats_pergame',
                           csv_name = 'per_game_stats.csv',
                           folder_write_to = 'clean_reg_season')
#Reg Season - Total Stats by Players
merge_csv_by_season_player(folder_from = 'season_stats_totals',
                           csv_name = 'totals_stats.csv',
                           folder_write_to = 'clean_reg_season')
#Reg Season - Advanced Stats by Players
merge_csv_by_season_player(folder_from = 'season_stats_advanced',
                           csv_name = 'advanced_stats.csv',
                           folder_write_to = 'clean_reg_season')
#Reg Season - MVP Shares
merge_csv_by_season_player(folder_from = 'award_stats',
                           csv_name = 'nba_mvp.csv',
                           folder_write_to = 'clean_reg_season')


##### Playoffs Stats

In [111]:
#Playoffs - Per Game Stats by Players
merge_csv_by_season_player(folder_from = 'playoffs_stats_pergame',
                           csv_name = 'per_game_stats.csv',
                           folder_write_to = 'clean_playoffs')
#Playoffs - Total Stats by Players
merge_csv_by_season_player(folder_from = 'playoffs_stats_totals',
                           csv_name = 'totals_stats.csv',
                           folder_write_to = 'clean_playoffs')
#Playoffs - Advanced Stats by Players
merge_csv_by_season_player(folder_from = 'playoffs_stats_advanced',
                           csv_name = 'advanced_stats.csv',
                           folder_write_to = 'clean_playoffs')

### Function to clean and merge player bio csv files separated by letter
___

In [146]:
def merge_csv_bios(folder_from: object,
                        csv_name: object, 
                        folder_write_to: object) -> object:
    #go to folder where csv files to be merged are
    os.chdir(folder_from)
    #create file list variable that will have all merged file data
    file_list = list()
    
    #looping through files in directory to prep data than append to master dataframe
    for file in os.listdir():
        if file.endswith('.csv'):
            df_2w = pd.read_csv(file, encoding="utf-8") 
            #split Player column into Player_Name and Player_ID
            new = df_2w["Player"].str.split("\\", expand = True)
            df_2w["Player_Name"]= new[0]
            df_2w["Player_ID"]= new[1]
            df_2w.drop(columns = ["Player"], inplace = True)
            # move the column to head of list using index, pop and insert
            cols = list(df_2w)
            cols.insert(0, cols.pop(cols.index('Player_Name')))
            cols.insert(1, cols.pop(cols.index('Player_ID')))
            # use loc to reorder
            df_2w = df_2w.loc[:, cols]
            #fill NAs
            df_2w['Colleges'].fillna('None', inplace=True)
            #convert height into inches and cm
            new_ht = df_2w['Ht'].str.split("-", expand = True)
            df_2w["Feet"]= new_ht[0]
            df_2w["Inches"]= new_ht[1]
            df_2w["Feet"] = pd.to_numeric(df_2w["Feet"])
            df_2w["Inches"]= pd.to_numeric(df_2w["Inches"])
            df_2w["Ht_Inches"] = (df_2w["Feet"]*12) + df_2w["Inches"]
            df_2w["Ht_cm"] = df_2w["Ht_Inches"]*2.54
            df_2w["Ht_m"] = df_2w["Ht_cm"]*.01
            df_2w["Ht_cm"] = df_2w["Ht_cm"].round(0)
            df_2w["Ht_m"] = df_2w["Ht_m"].round(2)
            #append to master dataframe to be written to csv                               
            file_list.append(df_2w)
    #merge all data in file_list, then create new file to write merged data to        
    merged_files = pd.concat(file_list, axis=0, ignore_index=True)
    #change to directory we are writing csv to
    back_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    os.chdir(back_dir)
    os.chdir(folder_write_to)
    #write csv
    merged_files.to_csv("all_" + csv_name)
    #go back to original directory
    os.chdir(back_dir)

### Using merge function for player bios

In [147]:
# using function for player bios
merge_csv_bios(folder_from = 'players_bios',
                           csv_name = 'players_bios.csv',
                           folder_write_to = 'clean_bios')

### Function for cleaning and merging shooting data files separated by player_ID. 
#### Note this group only includes players that played on or after 2000 

In [22]:
def merge_csv_player_shooting(
    folder_from: object,
    csv_name: object,
    folder_write_to: object) -> object:
    
    #go to folder where csv files to be merged are
    os.chdir(folder_from)
    #create file list variable that will have all merged file data
    file_list = list()
    
    #looping through files in directory to prep data than append to master dataframe
    for file in os.listdir():
        if file.endswith('.csv'):
            df_2w = pd.read_csv(file, encoding="utf-8")
            df_2w.rename(columns={'Unnamed: 0':'Season',
                                  'Unnamed: 1':'Age',
                                  'Unnamed: 2':'Tm',
                                  'Unnamed: 3':'Lg',
                                  'Unnamed: 4':'Pos',
                                  'Unnamed: 5':'G',
                                  'Unnamed: 6':'MP',
                                  'Unnamed: 7':'FG%',
                                  'Unnamed: 8':'Dist',
                                  'Unnamed: 9':'2PA/FGA',
                                  'Unnamed: 10':'Rate_FGA_distance_0-3_Feet',
                                  'Unnamed: 11':'Rate_FGA_distance_3-10_Feet',
                                  'Unnamed: 12':'Rate_FGA_distance_10-16_Feet',
                                  'Unnamed: 13':'Rate_FGA_distance_16-3ptLine_Feet',
                                  'Unnamed: 14':'3PA/FGA',
                                  'Unnamed: 15':'2ptFG%',
                                  'Unnamed: 16':'FG%_distance_0-3_Feet',
                                  'Unnamed: 17':'FG%_distance_3-10_Feet',
                                  'Unnamed: 18':'FG%_distance_10-16_Feet',
                                  'Unnamed: 19':'FG%_distance_16-3ptLine_Feet',
                                  'Unnamed: 20':'3ptFG%',
                                  '2-Pt Field Goals':'Rate_FG_Asstd',
                                  '2-Pt Field Goals.1':'Rate_of_dunks',
                                  '2-Pt Field Goals.2':'Total_dunks',
                                  '3-Pt Field Goals':'Rate_3ptFG_Asstd',
                                  '3-Pt Field Goals.1':'Rate_of_corner_3PA',
                                  '3-Pt Field Goals.2':'3P%_from_corner',
                                  '3-Pt Field Goals.3':'Heaves_attempted',
                                  '3-Pt Field Goals.4':'Heaves_made',
                                 }, inplace=True)
            new = df_2w['Season'].str.split("-", expand = True)
            new[1] = '20'+new[1]
            df_2w['Season'] = new[1]
            df_2w = df_2w[df_2w.Season.notnull()]
            df_2w['file'] = file
            new2 = df_2w['file'].str.split("_", expand = True)
            df_2w['Player_ID'] = new2[0]   
            df_2w = df_2w.drop('file', axis=1)
            #append to master dataframe to be written to csv                               
            file_list.append(df_2w)
    #merge all data in file_list, then create new file to write merged data to        
    merged_files = pd.concat(file_list, axis=0, ignore_index=True)
    #change to directory we are writing csv to
    back_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    os.chdir(back_dir)
    os.chdir(folder_write_to)
    #write csv
    merged_files.to_csv("all_seasons" + csv_name)
    #go back to original directory
    os.chdir(back_dir)

### Using merge function for shooting stats

In [23]:
# using function for player bios
merge_csv_player_shooting(folder_from = 'season_shooting',
                          csv_name = 'shooting.csv',
                          folder_write_to = 'clean_reg_season')

### Function for cleaning and merging salary data files separated by player_ID. 

In [34]:
def merge_csv_player_salaries(
    folder_from: object,
    csv_name: object,
    folder_write_to: object) -> object:
    
    #go to folder where csv files to be merged are
    os.chdir(folder_from)
    #create file list variable that will have all merged file data
    file_list = list()
    
    #looping through files in directory to prep data than append to master dataframe
    for file in os.listdir():
        if file.endswith('.csv'):
            salary_df = pd.read_csv(file, encoding="utf-8")
            new = salary_df['Season'].str.split("-", expand = True)
            new[1] = '20'+new[1]
            salary_df['Season'] = new[1]
            salary_df = salary_df[salary_df.Season.notnull()]
            salary_df['file'] = file
            new2 = salary_df['file'].str.split("_", expand = True)
            salary_df['Player_ID'] = new2[0]   
            salary_df = salary_df.drop('file', axis=1)
            #append to master dataframe to be written to csv                               
            file_list.append(salary_df)
    #merge all data in file_list, then create new file to write merged data to        
    merged_files = pd.concat(file_list, axis=0, ignore_index=True)
    #change to directory we are writing csv to
    back_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    os.chdir(back_dir)
    os.chdir(folder_write_to)
    #write csv
    merged_files.to_csv("all_seasons_" + csv_name)
    #go back to original directory
    os.chdir(back_dir)

### Using merge function for player salaries

In [35]:
merge_csv_player_salaries(folder_from = 'salaries',
                          csv_name = 'salaries.csv',
                          folder_write_to = 'clean_reg_season')